# Write CSV files from Particle sets
This notebook suggest a way of how to create CSV formatted files containing the particle track information originating from a **DaVis** particle set using **lvpyio**. In addition to the raw particle positions, the velocities are calculated and exported as well.

In [1]:
# Import necessary packages
import numpy as np
import lvpyio as lv
from csaps import CubicSmoothingSpline

## Define function to calculate particle velocity

In [2]:
# This is a function, which fits a single trajectory with a smoothing cubic spline. 
# Based on this spline, the first and second derivatives of the trajectories can be calculated.
# The first derivated is used to calculate the particle's velocity.
# Additionally the local curvature of the trajectory is calculated using both the first and second derivative.

def f_calc_velocity(track,dt,scaling, smooth_val):

    # get coordinates and rescale to meters
    x_raw = (track.particles[:]['x']*scaling.x.slope+scaling.x.offset)/1000
    y_raw = (track.particles[:]['y']*scaling.y.slope+scaling.y.offset)/1000
    z_raw = (track.particles[:]['z']*scaling.z.slope+scaling.z.offset)/1000
    
    # get track length
    length_track = np.size(x_raw)
    
    # Create arbitrary time vector. Absolute values not necessary here, only the spacing with dt
    t_raw=np.linspace(-length_track/2,length_track/2,length_track)*dt # create time vector for filter window
    
    # Fit x-data with Smoothing spline
    s = CubicSmoothingSpline(t_raw, x_raw, smooth=smooth_val, normalizedsmooth=True).spline
    # Calculate derivatives
    dx1 = s.derivative(nu=1)
    dx2 = s.derivative(nu=2)
    # Save velocity
    U = dx1(t_raw)
    
    # Fit y-data with Smoothing spline
    s = CubicSmoothingSpline(t_raw, y_raw, smooth=smooth_val, normalizedsmooth=True).spline
    # Calculate derivatives
    dy1 = s.derivative(nu=1)
    dy2 = s.derivative(nu=2)
    # Save velocity
    V = dy1(t_raw)
    
    # Fit z-data with Smoothing spline
    s = CubicSmoothingSpline(t_raw, z_raw, smooth=smooth_val, normalizedsmooth=True).spline
    # Calculate derivatives
    dz1 = s.derivative(nu=1)
    dz2 = s.derivative(nu=2)
    # Save velocity
    W = dz1(t_raw)
    
    return U, V, W

## Read particle data and add velocity information

In [3]:
# Read DaVis particle set
tracks_lv = lv.read_particles('data\\Tracks_Cylinder_Wake')

dt = tracks_lv.times()[1]-tracks_lv.times()[0] # retrieve dt of measurements

# initialize new list, which will contain the tracks with the added scalars
tracks_export = list()

# read scaling of positional data
scaling = tracks_lv.scales

# define scales of new scalars
scalar_scales = {"U": lv.Scale(1, 0, "m/s", "Spline velocity u"),"V": lv.Scale(1, 0, "m/s", "Spline velocity v"),"W": lv.Scale(1, 0, "m/s", "Spline velocity w")}

for ii in range(tracks_lv.track_count): # loop over all trajectories

    track_temp = tracks_lv.single_track(ii) # get trajectory information
    U, V, W = f_calc_velocity(track_temp,dt,scaling, 0.85) # calculate new scalars
    track_temp.scalars = {"U": U,"V": V,"W": W} # add scalars to track object
    
    tracks_export.append(track_temp) # add track to export list
    

## CSV Export

In [4]:

# check for output folder
import os
try:
    os.makedirs("data\\ExportCSV\\")
except FileExistsError:
    # directory already exists
    pass

# Specify column formatting for file export
# X Y Z U V W trackID
fmt = '%1.4f', '%1.4f', '%1.4f', '%1.4f', '%1.4f', '%1.4f', '%d'

for t in range(np.size(tracks_lv.times())): # loop over all time steps in order to create a single file per time step
    kk = 0
    
    for ii in range(np.size(tracks_export)): # loop over all trajectories
        
        # check if particle exists in the desired time step
        if (t >= tracks_export[ii].start) & (t < tracks_export[ii].start + np.size(tracks_export[ii].particles)):
            trackID = ii
            t_track = t-tracks_export[ii].start
            data_csv_temp = tracks_export[ii].particles["x"][t_track]*scaling.x.slope+scaling.x.offset # set first columns
            data_csv_temp = np.c_[data_csv_temp,tracks_export[ii].particles["y"][t_track]*scaling.y.slope+scaling.y.offset,
                                  tracks_export[ii].particles["z"][t_track]*scaling.z.slope+scaling.z.offset,
                                  tracks_export[ii].scalars["U"][t_track],
                                  tracks_export[ii].scalars["V"][t_track],
                                  tracks_export[ii].scalars["W"][t_track],
                                  trackID] # append other columns

            # write data array, which will then be exported
            if kk==0:
                data_csv = data_csv_temp
            else:
                data_csv = np.vstack([data_csv,data_csv_temp])
                
            kk = kk+1
      
    # save file 
    file_specifier = str(int(t)).zfill(3) # define leading zeros in file name   
    print("Exporting time-step: t" + file_specifier + ".txt")  
    np.savetxt("data\\ExportCSV\\ParticleData_t" + file_specifier + ".txt", data_csv, delimiter=" ", header='X[mm] Y[mm] Z[mm] U[m/s] V[m/s] W[m/s] trackID', fmt=fmt, comments='')

Exporting time-step: t000.txt
Exporting time-step: t001.txt
Exporting time-step: t002.txt
Exporting time-step: t003.txt
Exporting time-step: t004.txt
Exporting time-step: t005.txt
Exporting time-step: t006.txt
Exporting time-step: t007.txt
Exporting time-step: t008.txt
Exporting time-step: t009.txt
Exporting time-step: t010.txt
Exporting time-step: t011.txt
Exporting time-step: t012.txt
Exporting time-step: t013.txt
Exporting time-step: t014.txt
Exporting time-step: t015.txt
Exporting time-step: t016.txt
Exporting time-step: t017.txt
Exporting time-step: t018.txt
Exporting time-step: t019.txt
Exporting time-step: t020.txt
Exporting time-step: t021.txt
Exporting time-step: t022.txt
Exporting time-step: t023.txt
Exporting time-step: t024.txt
Exporting time-step: t025.txt
Exporting time-step: t026.txt
Exporting time-step: t027.txt
Exporting time-step: t028.txt
Exporting time-step: t029.txt
Exporting time-step: t030.txt
Exporting time-step: t031.txt
Exporting time-step: t032.txt
Exporting 